In [25]:
# このセルを実行（▶️をクリック）
import subprocess
from pathlib import Path
import os
import re
import time

# パスの設定
base_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分")
practice_path = base_path / "practice"
problems_path = practice_path / "problems"

# テンプレートを読み込む
template_path = base_path / "template.html"
with template_path.open("r", encoding="utf-8") as f:
    template = f.read()

# assignment.txtから解くべき問題を読み込む
assignment_path = practice_path / "assignment.txt"
try:
    assignment_input = assignment_path.read_text(encoding="utf-8").strip()
    if not assignment_input:
        print("⚠️ assignment.txtが空です。解くべき問題番号またはフォルダ名を指定してください。")
        print("例: 03 または problem_03_th_4_2_2 または 11-15")
        raise ValueError("assignment.txt is empty")
except FileNotFoundError:
    print("⚠️ assignment.txtが見つかりません。")
    raise

def process_single_problem(problem_num_str):
    """単一の問題番号を処理する関数"""
    # 番号を2桁の文字列に正規化（例: "3" -> "03", "03" -> "03"）
    problem_num = problem_num_str.zfill(2)
    
    # problem_XX_* というパターンでマッチするフォルダを探す
    pattern = f"problem_{problem_num}_"
    matching_folders = []
    for p in problems_path.iterdir():
        if p.is_dir() and p.name.startswith(pattern):
            matching_folders.append(p.name)
    
    if len(matching_folders) == 0:
        print(f"⚠️ 問題番号 {problem_num} に対応する問題フォルダが見つかりません。")
        print("利用可能な問題フォルダ:")
        for p in sorted(problems_path.iterdir()):
            if p.is_dir():
                print(f"  - {p.name}")
        raise FileNotFoundError(f"Problem folder not found for number: {problem_num}")
    elif len(matching_folders) > 1:
        print(f"⚠️ 問題番号 {problem_num} に対応する問題フォルダが複数見つかりました:")
        for folder in matching_folders:
            print(f"  - {folder}")
        print("完全なフォルダ名を指定してください。")
        raise ValueError(f"Multiple folders found for number: {problem_num}")
    else:
        assignment_text = matching_folders[0]
        print(f"📌 番号 {problem_num} に対応する問題フォルダ: {assignment_text}")
    
    # 問題フォルダのパス
    problem_folder = problems_path / assignment_text
    if not problem_folder.exists():
        print(f"⚠️ 問題フォルダが見つかりません: {problem_folder}")
        print("利用可能な問題フォルダ:")
        for p in sorted(problems_path.iterdir()):
            if p.is_dir():
                print(f"  - {p.name}")
        raise FileNotFoundError(f"Problem folder not found: {problem_folder}")
    
    # 問題と解答を読み込む
    problem_file = problem_folder / "problem.md"
    solution_file = problem_folder / "solution.md"
    
    problem_text = ""
    if problem_file.exists():
        problem_text = problem_file.read_text(encoding="utf-8")
    else:
        print(f"⚠️ problem.mdが見つかりません: {problem_file}")
    
    solution_text = ""
    if solution_file.exists():
        solution_text = solution_file.read_text(encoding="utf-8")
    else:
        print(f"⚠️ solution.mdが見つかりません: {solution_file}")
        raise FileNotFoundError(f"Solution file not found: {solution_file}")
    
    # 問題番号を抽出（フォルダ名から推測）
    # フォルダ名から番号部分を抽出（例: "problem_03_th_4_2_2" -> "03"）
    match = re.match(r"problem_(\d+)_", assignment_text)
    if match:
        problem_number = match.group(1).zfill(2)  # 2桁に正規化
    else:
        # フォールバック: 従来の方法
        problem_number = assignment_text.replace("problem_", "").replace("_", " ").title()
    
    instruction_header = f"""まず、@homework/微分積分/practice/output/solution.htmlの名前を適切に変更し、@homework/微分積分/practice/archive/に移動させた後、以下の情報を参照して解答を作成してください。

【参照ファイル】
- @homework/微分積分/practice/problems/{assignment_text}/problem.md: 問題が記載されています
- @homework/微分積分/practice/problems/{assignment_text}/solution.md: 各問題の解答が記載されています
- @homework/微分積分/practice/assignment.txt: 今回解くべき問題は「{assignment_text}」です

【問題と解答の内容】
--- problem.md（問題） ---
{problem_text}
--- problem.md ここまで ---

--- solution.md（解答） ---
{solution_text}
--- solution.md ここまで ---

【指示】
`assignment.txt`に記載された問題「{assignment_text}」に対応する問題を`problem.md`から見つけ、その問題の解答を`solution.md`を参照しながら、初学者にも超わかりやすく丁寧に解説してください。

解答を作成する際は：
1. `solution.md`の解答内容を基に、計算過程を省略せずにすべて詳しく説明します
2. 各ステップで「なぜその計算をするのか」「どの公式を使っているのか」を明確に説明します
3. 数式の変形の理由を必ず文章で補足します
4. 初学者がつまずきやすいポイントには`note`ブロックで補足説明を加えます

出力は次のテンプレートと文体を厳守した HTML を @homework/微分積分/practice/output/solution.html に保存すること。

【テンプレート仕様】
- 下記 HTML をそのまま骨格として使用し、問題タイトル・本文・数式・説明だけを差し替える。
- class 名、タグ階層、CSS を一切変更しない。不要な要素を追加しない。
- MathJax はテンプレートの設定のまま利用し、数式は `$...$`（行内）と `$$...$$`（ディスプレイ）を使用。
"""
    
    instruction_footer = f"""【執筆ルール】
- `assignment.txt`に記載された問題のみを処理する。`h1`は「微分積分 {problem_number} 解答」とし、`h2`は「【解答】」固定で始める。問題文ブロックは`div.problem`内に`【問題】`と対象関数を必ず記載する。
- 問題文ブロック（`div.problem`）と`h2`の「【解答】」の間に、必ず「【方針】」セクションを追加します。このセクションは`div.strategy`で囲み、`div.strategy-title`に「【方針】」というタイトルを入れます。
- 方針セクション内では、まず最初に「必要な知識・テクニック」の紹介セクションを追加します。このセクションは`div.knowledge`で囲み、`div.knowledge-title`に「必要な知識・テクニック」というタイトルを入れます。このセクションでは、問題を解くために必要な公式・定理・テクニックを箇条書きで列挙し、それぞれについて簡潔に特徴や使用場面を説明します（例：「積の微分法：2つの関数の積を微分する際に使用する。$(fg)' = f'g + fg'$の形で表される。」）。このセクションにより、読者が解答を読む前に必要な前提知識を把握できるようにします。
- その後に、`div.flowchart`を使用してフローチャートを作成します。フローチャートは以下の構造で作成します：
  * 各ステップは`div.flowchart-step`で表現し、適切なクラス（`start`：開始、`process`：処理、`decision`：条件分岐、`end`：終了）を付与します
  * ステップ間には`div.flowchart-arrow`で矢印を挿入します
  * 各ステップ内には`div.flowchart-label`でステップ番号やタイトルを表示し、その下にステップの説明を記載します
  * フローチャートは解答全体の流れを視覚的に表現し、以下のような要素を含めます：問題の理解、必要な公式・定理の確認、計算手順、判定基準、結果の検証など
  * 条件分岐がある場合は、判定結果（Yes/No）によって異なる処理が分かれることを示します
  * フローチャートは縦方向に流れるように配置し、各ステップは簡潔に（1〜2行程度）記述します
  * フローチャートのHTML構造例は`template.html`のコメント内に記載されていますので、それを参照してください
- `h2`の「【解答】」の後、最初の`step`ブロックの前に、解答全体の方針を説明する段落を必ず追加します。この段落では、どのような手順・アプローチで問題を解いていくかを簡潔に概説します（例：「この問題では、まず〜を行い、次に〜を求め、最後に〜を確認します。」）。全体の方針を示すことで、読者が解答の流れを事前に把握できるようにします。ただし、この段落はフローチャートで視覚的に表現されている内容を文章で補足する形にします。
- 各`step`ブロックは`solution.md`の解答を基に、計算過程を省略せずに詳しく説明します。`step-title`は「（1）〜」形式で連番を振ります。段落の語尾は「〜します。」で統一します。各ステップはフローチャートで示された手順に対応するように構成します。
- 数式を示したら必ず直後に文章で解説し、式だけで終わらせません。`formula`ブロックの前後に説明段落を入れます。
- `solution.md`に記載されている計算過程や判定基準はすべて含め、省略しないでください。
- `note`、`calculation`、`verification`の各ブロックもテンプレートのスタイルで使用し、初学者が理解しやすいように補足説明を加えます。
- 検証セクションには少なくとも2つの確認手順を段落で述べ、最後に「以上より求めた結果の正しさを確認しました。」という文で締めます。
- 末尾に追加のスクリプトやlive-serverのコードは禁止です。

以上を厳守し、`solution.md`の解答内容を基に、丁寧な微分積分解説を生成してください。"""
    
    command = (
        instruction_header
        + "\n--- HTML テンプレート（そのまま利用） ---\n"
        + template
        + "\n--- テンプレートここまで ---\n\n"
        + instruction_footer
    )
    
    return command, assignment_text, problem_folder

# 範囲指定かどうかを判定（例: "11-15"）
range_match = re.match(r"^(\d+)-(\d+)$", assignment_input)
if range_match:
    # 範囲指定の場合
    start_num = int(range_match.group(1))
    end_num = int(range_match.group(2))
    
    if start_num > end_num:
        print(f"⚠️ 範囲指定が不正です: {start_num} > {end_num}")
        raise ValueError("Invalid range: start > end")
    
    print(f"📋 範囲指定を検出: {start_num} から {end_num} まで ({end_num - start_num + 1}問)")
    
    # outputディレクトリを作成
    output_dir = practice_path / "output"
    output_dir.mkdir(exist_ok=True)
    
    # archiveディレクトリを作成
    archive_dir = practice_path / "archive"
    archive_dir.mkdir(exist_ok=True)
    
    # 各問題を順次処理
    for problem_num in range(start_num, end_num + 1):
        problem_num_str = str(problem_num)
        print("\n" + "="*60)
        print(f"🔄 問題 {problem_num_str} を処理中... ({problem_num - start_num + 1}/{end_num - start_num + 1})")
        print("="*60)
        
        try:
            command, assignment_text, problem_folder = process_single_problem(problem_num_str)
            
            # 1回目のプロンプトを送信
            subprocess.run('pbcopy', text=True, input=command, check=True)
            if problem_num == start_num:
                # 最初の問題のみComposerを開く
                subprocess.run(['osascript', '-e', '''
                tell application "Cursor"
                    activate
                    delay 0.3
                end tell
                tell application "System Events"
                    keystroke "i" using command down
                    delay 0.3
                    keystroke "v" using command down
                    delay 0.3
                    keystroke return
                    delay 1.0
                end tell
                '''], check=True)
            else:
                # 2回目以降は既に開いているComposerにペーストするだけ
                subprocess.run(['osascript', '-e', '''
                tell application "Cursor"
                    activate
                    delay 0.3
                end tell
                tell application "System Events"
                    keystroke "v" using command down
                    delay 0.3
                    keystroke return
                    delay 1.0
                end tell
                '''], check=True)
            print(f'✅ 問題 {problem_num_str}: プロンプト 1/2 を送信しました...')
            
            # 2回目のプロンプトを送信（すでにComposerが開いているので、ペーストとエンターのみ）
            second_prompt = "@solution.html の内容を入念に調べ間違いがないか確認して"
            subprocess.run('pbcopy', text=True, input=second_prompt, check=True)
            subprocess.run(['osascript', '-e', '''
            tell application "Cursor"
                activate
                delay 0.3
            end tell
            tell application "System Events"
                keystroke "v" using command down
                delay 0.3
                keystroke return
            end tell
            '''], check=True)
            print(f'✅ 問題 {problem_num_str}: プロンプト 2/2 を送信しました...')
            print(f'✅ 問題 {problem_num_str}: プロンプトを2回送信完了 - 微分積分問題解説生成中...')
            print(f'📝 問題: {assignment_text}')
            print(f'📁 問題フォルダ: {problem_folder}')
            
            # 次の問題に進む前に少し待機（連続送信を避けるため）
            if problem_num < end_num:
                print(f'\n⏳ 次の問題に進む前に3秒待機します...')
                time.sleep(3)
        
        except Exception as e:
            print(f'❌ 問題 {problem_num_str} でエラー: {e}')
            print('手動: Cmd+I → 下記を貼り付け')
            print(f'\n1回目:\n{command if "command" in locals() else "コマンド生成エラー"}')
            print(f'\n2回目:\n@solution.html の内容を入念に調べて読みやすい解答となるように整理して')
            # エラーが発生しても次の問題に進む
            continue
    
    print('\n' + "="*60)
    print(f'✅ 全問題の処理が完了しました ({start_num}-{end_num})')
    print("="*60)

else:
    # 単一の問題番号またはフォルダ名指定の場合
    assignment_text = assignment_input
    
    # 数字のみの場合は番号指定として扱う
    if assignment_input.isdigit() or (assignment_input.startswith("0") and assignment_input[1:].isdigit()):
        command, assignment_text, problem_folder = process_single_problem(assignment_input)
    else:
        # フォルダ名指定の場合
        problem_folder = problems_path / assignment_text
        if not problem_folder.exists():
            print(f"⚠️ 問題フォルダが見つかりません: {problem_folder}")
            print("利用可能な問題フォルダ:")
            for p in sorted(problems_path.iterdir()):
                if p.is_dir():
                    print(f"  - {p.name}")
            raise FileNotFoundError(f"Problem folder not found: {problem_folder}")
        
        # 問題と解答を読み込む
        problem_file = problem_folder / "problem.md"
        solution_file = problem_folder / "solution.md"
        
        problem_text = ""
        if problem_file.exists():
            problem_text = problem_file.read_text(encoding="utf-8")
        else:
            print(f"⚠️ problem.mdが見つかりません: {problem_file}")
        
        solution_text = ""
        if solution_file.exists():
            solution_text = solution_file.read_text(encoding="utf-8")
        else:
            print(f"⚠️ solution.mdが見つかりません: {solution_file}")
            raise FileNotFoundError(f"Solution file not found: {solution_file}")
        
        # 問題番号を抽出（フォルダ名から推測）
        match = re.match(r"problem_(\d+)_", assignment_text)
        if match:
            problem_number = match.group(1).zfill(2)
        else:
            problem_number = assignment_text.replace("problem_", "").replace("_", " ").title()
        
        instruction_header = f"""まず、@homework/微分積分/practice/output/solution.htmlの名前を適切に変更し、@homework/微分積分/practice/archive/に移動させた後、以下の情報を参照して解答を作成してください。

【参照ファイル】
- @homework/微分積分/practice/problems/{assignment_text}/problem.md: 問題が記載されています
- @homework/微分積分/practice/problems/{assignment_text}/solution.md: 各問題の解答が記載されています
- @homework/微分積分/practice/assignment.txt: 今回解くべき問題は「{assignment_text}」です

【問題と解答の内容】
--- problem.md（問題） ---
{problem_text}
--- problem.md ここまで ---

--- solution.md（解答） ---
{solution_text}
--- solution.md ここまで ---

【指示】
`assignment.txt`に記載された問題「{assignment_text}」に対応する問題を`problem.md`から見つけ、その問題の解答を`solution.md`を参照しながら、初学者にも超わかりやすく丁寧に解説してください。

解答を作成する際は：
1. `solution.md`の解答内容を基に、計算過程を省略せずにすべて詳しく説明します
2. 各ステップで「なぜその計算をするのか」「どの公式を使っているのか」を明確に説明します
3. 数式の変形の理由を必ず文章で補足します
4. 初学者がつまずきやすいポイントには`note`ブロックで補足説明を加えます

出力は次のテンプレートと文体を厳守した HTML を @homework/微分積分/practice/output/solution.html に保存すること。

【テンプレート仕様】
- 下記 HTML をそのまま骨格として使用し、問題タイトル・本文・数式・説明だけを差し替える。
- class 名、タグ階層、CSS を一切変更しない。不要な要素を追加しない。
- MathJax はテンプレートの設定のまま利用し、数式は `$...$`（行内）と `$$...$$`（ディスプレイ）を使用。
"""
        
        instruction_footer = f"""【執筆ルール】
- `assignment.txt`に記載された問題のみを処理する。`h1`は「微分積分 {problem_number} 解答」とし、`h2`は「【解答】」固定で始める。問題文ブロックは`div.problem`内に`【問題】`と対象関数を必ず記載する。
- 問題文ブロック（`div.problem`）と`h2`の「【解答】」の間に、必ず「【方針】」セクションを追加します。このセクションは`div.strategy`で囲み、`div.strategy-title`に「【方針】」というタイトルを入れます。
- 方針セクション内では、まず最初に「必要な知識・テクニック」の紹介セクションを追加します。このセクションは`div.knowledge`で囲み、`div.knowledge-title`に「必要な知識・テクニック」というタイトルを入れます。このセクションでは、問題を解くために必要な公式・定理・テクニックを箇条書きで列挙し、それぞれについて簡潔に特徴や使用場面を説明します（例：「積の微分法：2つの関数の積を微分する際に使用する。$(fg)' = f'g + fg'$の形で表される。」）。このセクションにより、読者が解答を読む前に必要な前提知識を把握できるようにします。
- その後に、`div.flowchart`を使用してフローチャートを作成します。フローチャートは以下の構造で作成します：
  * 各ステップは`div.flowchart-step`で表現し、適切なクラス（`start`：開始、`process`：処理、`decision`：条件分岐、`end`：終了）を付与します
  * ステップ間には`div.flowchart-arrow`で矢印を挿入します
  * 各ステップ内には`div.flowchart-label`でステップ番号やタイトルを表示し、その下にステップの説明を記載します
  * フローチャートは解答全体の流れを視覚的に表現し、以下のような要素を含めます：問題の理解、必要な公式・定理の確認、計算手順、判定基準、結果の検証など
  * 条件分岐がある場合は、判定結果（Yes/No）によって異なる処理が分かれることを示します
  * フローチャートは縦方向に流れるように配置し、各ステップは簡潔に（1〜2行程度）記述します
  * フローチャートのHTML構造例は`template.html`のコメント内に記載されていますので、それを参照してください
- `h2`の「【解答】」の後、最初の`step`ブロックの前に、解答全体の方針を説明する段落を必ず追加します。この段落では、どのような手順・アプローチで問題を解いていくかを簡潔に概説します（例：「この問題では、まず〜を行い、次に〜を求め、最後に〜を確認します。」）。全体の方針を示すことで、読者が解答の流れを事前に把握できるようにします。ただし、この段落はフローチャートで視覚的に表現されている内容を文章で補足する形にします。
- 各`step`ブロックは`solution.md`の解答を基に、計算過程を省略せずに詳しく説明します。`step-title`は「（1）〜」形式で連番を振ります。段落の語尾は「〜します。」で統一します。各ステップはフローチャートで示された手順に対応するように構成します。
- 数式を示したら必ず直後に文章で解説し、式だけで終わらせません。`formula`ブロックの前後に説明段落を入れます。
- `solution.md`に記載されている計算過程や判定基準はすべて含め、省略しないでください。
- `note`、`calculation`、`verification`の各ブロックもテンプレートのスタイルで使用し、初学者が理解しやすいように補足説明を加えます。
- 検証セクションには少なくとも2つの確認手順を段落で述べ、最後に「以上より求めた結果の正しさを確認しました。」という文で締めます。
- 末尾に追加のスクリプトやlive-serverのコードは禁止です。

以上を厳守し、`solution.md`の解答内容を基に、丁寧な微分積分解説を生成してください。"""
        
        command = (
            instruction_header
            + "\n--- HTML テンプレート（そのまま利用） ---\n"
            + template
            + "\n--- テンプレートここまで ---\n\n"
            + instruction_footer
        )
    
    try:
        # outputディレクトリを作成
        output_dir = practice_path / "output"
        output_dir.mkdir(exist_ok=True)
        
        # archiveディレクトリを作成
        archive_dir = practice_path / "archive"
        archive_dir.mkdir(exist_ok=True)
        
        # 1回目のプロンプトを送信
        subprocess.run('pbcopy', text=True, input=command, check=True)
        subprocess.run(['osascript', '-e', '''
        tell application "Cursor"
            activate
            delay 0.3
        end tell
        tell application "System Events"
            keystroke "i" using command down
            delay 0.3
            keystroke "v" using command down
            delay 0.3
            keystroke return
            delay 1.0
        end tell
        '''], check=True)
        print('✅ プロンプト 1/2 を送信しました...')
        
        # 2回目のプロンプトを送信（すでにComposerが開いているので、ペーストとエンターのみ）
        second_prompt = "@solution.html の内容を入念に調べ間違いがないか確認して"
        subprocess.run('pbcopy', text=True, input=second_prompt, check=True)
        subprocess.run(['osascript', '-e', '''
        tell application "Cursor"
            activate
            delay 0.3
        end tell
        tell application "System Events"
            keystroke "v" using command down
            delay 0.3
            keystroke return
        end tell
        '''], check=True)
        print('✅ プロンプト 2/2 を送信しました...')
        print('✅ プロンプトを2回送信完了 - 微分積分問題解説生成中...')
        print(f'\n📝 問題: {assignment_text}')
        print(f'📁 問題フォルダ: {problem_folder}')
    except Exception as e:
        print(f'❌ エラー: {e}')
        print('手動: Cmd+I → 下記を貼り付け')
        print(f'\n1回目:\n{command}')
        print(f'\n2回目:\n@solution.html の内容を入念に調べて読みやすい解答となるように整理して')

📌 番号 04 に対応する問題フォルダ: problem_04_limit_4_2_3
⚠️ problem.mdが見つかりません: /Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/practice/problems/problem_04_limit_4_2_3/problem.md
✅ プロンプト 1/2 を送信しました...
✅ プロンプト 2/2 を送信しました...
✅ プロンプトを2回送信完了 - 微分積分問題解説生成中...

📝 問題: problem_04_limit_4_2_3
📁 問題フォルダ: /Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/practice/problems/problem_04_limit_4_2_3
